<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Generate synthetic data
data, _ = make_blobs(n_samples=1000, centers=2, cluster_std=1.0)
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

X_train, X_temp = train_test_split(data, test_size=0.4, random_state=42)
X_val, X_test = train_test_split(X_temp, test_size=0.5, random_state=42)
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

# Define the autoencoder model
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

model = Autoencoder(input_dim=2)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# Training loop with validation
for epoch in range(50):
    model.train()
    outputs = model(X_train)
    loss = criterion(outputs, X_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()

    if epoch % 10 == 0:
        model.eval()
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, X_val)
        print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

# Testing for anomaly detection
model.eval()
test_outputs = model(X_test)
reconstruction_errors = torch.mean((test_outputs - X_test) ** 2, axis=1)
threshold = torch.mean(reconstruction_errors) + 2 * torch.std(reconstruction_errors)
anomalies = reconstruction_errors > threshold

print(f"Anomalies detected: {torch.sum(anomalies).item()}")